<a href="https://colab.research.google.com/github/Bhardwaj-Saurabh/Agents_Design_Pattern/blob/master/Tool_Calling_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [302]:
# !pip install -U langchain-google-genai langchain-openai

In [303]:
import json

def calculator(a: float | int,
               b: float | int,
               operation: str):
    """
    Calculator to perform mathamatical operation between two numbers.

    a: float | int - first number
    b: float | int - second number
    operation: str - mathamatical operation needed

    return: result in json format
    """
    if operation == 'add':
        return json.dumps({"output": a + b})
    elif operation == 'multiply':
        return json.dumps({"output": a * b})
    elif operation == 'subtract':
        return json.dumps({"output": a - b})
    elif operation == 'division':
        return json.dumps({"output": a / b})
    else:
        return json.dumps({"output": None})

In [304]:
calculator

<function __main__.calculator(a: float | int, b: float | int, operation: str)>

In [305]:
calculator(2, 6, "multiply")

'{"output": 12}'

In [306]:
import os
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [307]:
os.environ['GOOGLE_API_KEY'] = GOOGLE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [308]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [309]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
)

In [310]:
# Define the System Prompt as a constant
TOOL_SYSTEM_PROMPT = """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>}
</tool_call>

Here are the available tools:

<tools>
CALCULATOR:
"name": "calculator","description": "Calculator to perform mathamatical operation between two numbers. a: float | int - first number b: float | int - second number operation: str - mathamatical operation needed (must be one of the followings: add, multiply, subtract, division), return: result",

</tools>
"""

In [311]:
system_message = (
        "system",
        TOOL_SYSTEM_PROMPT
    )

user_message = (
        "human",
        "give me multiplication result 3 and 8.9"
    )

history = [system_message, user_message]

In [312]:
output = llm.invoke(history)

In [313]:
print(output.content)

```xml
<tool_call>
{"name": "calculator","arguments": {"a": 3, "b": 8.9, "operation": "multiply"}}
</tool_call>
```


In [314]:
import re
import json

def parse_tool_call_str(tool_call_str: str):
    pattern = r'</?tool_call>'
    clean_tags = re.sub(pattern, '', tool_call_str)

    if 'xml' in clean_tags:
        clean_tags = clean_tags.split('\n\n')[1]

    try:
        tool_call_json = json.loads(clean_tags)
        return tool_call_json
    except json.JSONDecodeError:
        return clean_tags
    except Exception as e:
        return None

In [315]:
parsed_output = parse_tool_call_str(output.content)
parsed_output

{'name': 'calculator',
 'arguments': {'a': 3, 'b': 8.9, 'operation': 'multiply'}}

In [316]:
result = calculator(**parsed_output["arguments"])

In [317]:
result

'{"output": 26.700000000000003}'

In [318]:
history.append(
    ("assistant",
    f"Answer of the query: {result}"))

In [319]:
history

[('system',
  '\nYou are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.\nYou may call one or more functions to assist with the user query. Don\'t make assumptions about what values to plug\ninto functions. Pay special attention to the properties \'types\'. You should use those types as in a Python dict.\nFor each function call return a json object with function name and arguments within <tool_call></tool_call> XML tags as follows:\n\n<tool_call>\n{"name": <function-name>,"arguments": <args-dict>}\n</tool_call>\n\nHere are the available tools:\n\n<tools> \nCALCULATOR:\n"name": "calculator","description": "Calculator to perform mathamatical operation between two numbers. a: float | int - first number b: float | int - second number operation: str - mathamatical operation needed (must be one of the followings: add, multiply, subtract, division), return: result",\n\n</tools>\n'),
 ('human', 'give me multiplication result 3 and 8.9'),


In [320]:
history.append(
    ("human",
    "Once you have answer to the query in the histroy. Provide the final answer with query in a mardown format "))

In [321]:
response = llm.invoke(history)

In [322]:
from IPython.display import Markdown

Markdown(response.content)

```markdown
**Query:** give me multiplication result 3 and 8.9

**Answer:** 26.700000000000003
```